### code to compare mjo teleconnection pattern between DLWPs and ERA5

In [1]:
import xarray as xr
import numpy as np
import os
from datetime import datetime, timedelta

ds = xr.open_zarr(
  'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',

  chunks=None,
  storage_options=dict(token='anon'),
)

#### 1. Pick MJO phase 3 samples and prepare for validation data 

In [3]:
#===== MJO phase 3 cases picked from ERA5 =====#
fname = './historical-MJO-P3_cases.txt'
dates = []

# Open the file and read the lines
with open(fname, 'r') as file:
    for line in file:
        if not line.startswith("#"):
            date = line.strip()
            dates.append(date)

# Print the list of dates
print(dates)

['2000-11-17', '2001-01-16', '2001-01-30', '2002-01-19', '2002-11-08', '2002-12-23', '2003-12-01', '2004-01-29', '2006-01-13', '2006-02-27', '2006-12-24', '2007-02-22', '2007-12-13', '2008-02-03', '2008-11-16', '2009-02-25', '2009-11-06', '2009-12-30', '2010-02-18', '2010-11-28', '2011-11-01', '2012-02-29', '2012-11-01', '2012-12-26', '2013-02-14', '2014-02-06', '2015-12-02', '2016-11-24', '2017-02-28', '2017-11-21', '2018-01-01', '2018-02-28', '2018-12-12', '2020-02-29', '2021-01-04', '2022-02-10', '2022-11-07', '2023-01-27']


In [ ]:
#===== create initial data for Pangu from ERA-5=====#
fpath = '../data/ERA5/initial_hist-MJO-P3_Pangu/'
plev0 = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50] # levels used in Pangu

for date in dates:
    print(date)

    time0 = date + "T00"
    #========= surface =========#
    vname_srf = [
        'mean_sea_level_pressure',
        '10m_u_component_of_wind',
        '10m_v_component_of_wind',
        '2m_temperature'
    ]

    v_srf = ds[vname_srf].sel(time = time0)
    v_srf = v_srf.to_array().to_numpy()

    fname = fpath + 'input_srf_' + date + '.npy'
    np.save(fname, v_srf)
    #========= upper ===========#
    vname_upper = [
        'geopotential',
        'specific_humidity',
        'temperature',
        'u_component_of_wind',
        'v_component_of_wind'
    ]

    v_upper = ds[vname_upper].sel(time = time0, level = plev0)
    v_upper = v_upper.to_array().to_numpy()

    fname = fpath + 'input_upper_' + date + '.npy'
    np.save(fname, v_upper)

2000-11-17
2001-01-16
2001-01-30
2002-01-19
2002-11-08
2002-12-23
2003-12-01
2004-01-29
2006-01-13
2006-02-27
2006-12-24
2007-02-22
2007-12-13
2008-02-03
2008-11-16
2009-02-25
2009-11-06
2009-12-30
2010-02-18
2010-11-28
2011-11-01
2012-02-29
2012-11-01
2012-12-26
2013-02-14
2014-02-06
2015-12-02
2016-11-24
2017-02-28
2017-11-21
2018-01-01
2018-02-28
2018-12-12
2020-02-29
2021-01-04
2022-02-10
2022-11-07
2023-01-27


In [14]:
#===== create initial data for NeuralGCM from ERA-5=====#
# No need to store that, it is done in NeuralGCM code

In [3]:
#===== create validation data from ERA-5 =====#
fpath = './ERA5_subset/validation_hist-MJO-P3/'

plev0 = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50] # levels used in Pangu

for date in dates:
    # #========= cut 30 days =========#
    print(date)
    temp = datetime.strptime(date, "%Y-%m-%d")

    date_range = []
    for i in range(31):
        date_range.append( (temp+timedelta(days=i)).strftime("%Y-%m-%d"))
    
    # Convert back to string
    date_range = [date + "T00" for date in date_range]

    #========= surface =========#
    vname_srf = [
        'mean_sea_level_pressure',
        '10m_u_component_of_wind',
        '10m_v_component_of_wind',
        '2m_temperature'
    ]

    v_srf = ds[vname_srf].sel(time = date_range)

    fname = fpath + 'era5_srf_' + date + '_30days.nc'
    v_srf.to_netcdf(fname)
    #========= upper ===========#
    vname_upper = [
        'geopotential'#,
        # 'specific_humidity',
        # 'temperature',
        # 'u_component_of_wind',
        # 'v_component_of_wind'
    ]
    v_upper = ds[vname_upper].sel(time = date_range, level = plev0)

    fname = fpath + 'era5_Z_' + date + '_30days.nc'
    v_upper.to_netcdf(fname)

2000-11-17
2001-01-16
2001-01-30
2002-01-19
2002-11-08
2002-12-23
2003-12-01
2004-01-29
2006-01-13
2006-02-27
2006-12-24
2007-02-22
2007-12-13
2008-02-03
2008-11-16
2009-02-25
2009-11-06
2009-12-30
2010-02-18
2010-11-28
2011-11-01
2012-02-29
2012-11-01
2012-12-26
2013-02-14
2014-02-06
2015-12-02
2016-11-24
2017-02-28
2017-11-21
2018-01-01
2018-02-28
2018-12-12
2020-02-29
2021-01-04
2022-02-10
2022-11-07
2023-01-27


In [ ]:
#===== generate ensemble mean data as the intial condition of ens-mean experiment =====#
fpath = '../data/ERA5/initial_hist-MJO-P3_Pangu/'

files_srf = [os.path.join(fpath, f) for f in os.listdir(fpath) if 'srf' in f and f.endswith('.npy')]
files_upper = [os.path.join(fpath, f) for f in os.listdir(fpath) if 'upper' in f and f.endswith('.npy')]

def calculate_mean(files):
    mean_data = None
    count = 0
    
    for file in files:
        data = np.load(file)
        if mean_data is None:
            mean_data = data
        else:
            mean_data += data
        count += 1
    
    mean_data /= count
    return mean_data

mean_srf = calculate_mean(files_srf)
mean_upper = calculate_mean(files_upper)

np.save(fpath + 'input_srf_ens-mean.npy', mean_srf)
np.save(fpath + 'input_upper_ens-mean.npy', mean_upper)


In [ ]:
#===== MJO phase 3 cases picked from ERA5 =====#
fname = './historical-MJO-P3_cases_2017beyond.txt'
dates = []

# Open the file and read the lines
with open(fname, 'r') as file:
    for line in file:
        if not line.startswith("#"):
            date = line.strip()
            dates.append(date)

# Print the list of dates
print(dates)

fpath = '../data/ERA5/initial_hist-MJO-P3_Pangu/'

files_upper = [os.path.join(fpath, f) for f in os.listdir(fpath) if 'upper' in f and f.endswith('.npy') and any(date in f for date in dates) ]
files_srf = [os.path.join(fpath, f) for f in os.listdir(fpath) if 'srf' in f and f.endswith('.npy')and any(date in f for date in dates) ]

def calculate_mean(files):
    mean_data = None
    count = 0
    
    for file in files:
        data = np.load(file)
        if mean_data is None:
            mean_data = data
        else:
            mean_data += data
        count += 1
    
    mean_data /= count
    return mean_data

mean_srf = calculate_mean(files_srf)
mean_upper = calculate_mean(files_upper)

np.save(fpath + 'input_srf_ens-mean_2017beyond.npy', mean_srf)
np.save(fpath + 'input_upper_ens-mean_2017beyond.npy', mean_upper)

['2018-01-01', '2018-02-28', '2018-12-12', '2020-02-29', '2021-01-04', '2022-02-10', '2022-11-07', '2023-01-27']
